#### **Gruppo 5.b**: Caponi Marco (matricola: 508773) - Ceneda Gianluca (matricola: 488257)

# ANALISI E REVISIONE DEL PROGETTO LARSPLITTING 2D 

## CLASSE REFACTORING: congruence

Variabili utili per testare il funzionamento


In [1]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads


In [2]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);    #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);   #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];             #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

## Versione iniziale di congruence

funzione che prende in ingresso un modello di Lar, restituendo una funzione di base denominata hcat che concatena due array lungo due dimensioni.

In [3]:
function congruence(model)
	W,EW = model
	# congruent vertices
	balltree = NearestNeighbors.BallTree(W)
	r = 0.0000000001
	near = Array{Any}(undef, size(W,2))
	for k=1:size(W,2)
		near[k] = union([NearestNeighbors.inrange(balltree, W[:,k], r, true)]...)
	end
	near = map(sort,near)  # check !!!
	for k=1:size(W,2)
		W[:,k] = W[:,near[k][1]]
	end
	pointidx = [ near[k][1] for k=1:size(W,2) ]  # check !!
	invidx = OrderedDict(zip(1:length(pointidx), pointidx))
	V = [W[:,k] for k=1:length(pointidx)]
	# congruent edges
	EV = []
	for e in (EW)
		newedge = [invidx[e[1]],invidx[e[2]]]
		if newedge[1] !== newedge[2]
			push!(EV,newedge)
		end
	end
	EV = [EV[h] for h=1:length(EV) if length(EV[h])==2]
	EV = convert(Lar.Cells, EV)

	return hcat(V...),EV
end




congruence (generic function with 1 method)

In [12]:
@btime congruence((V, EV))       #36,698 μs

  36.698 μs (161 allocations: 8.92 KiB)


([0.0 1.0 … 0.0 2.0; 0.0 0.0 … 1.0 1.0], [[1, 2], [2, 3], [3, 4], [4, 1], [1, 5]])

La funzione è type stable in quanto ritorna la stringa:

Body::Tuple{Array,Any}

## Versione modificata di congruence

Siamo riusciti ad avere notevoli miglioramenti attraverso l'utilizzo delle macro **@inbounds** e **@threads**

In [6]:
using Base.Threads
function congruenceMOD(model)
    W,EW = model
    n = size(W,2)
    balltree = NearestNeighbors.BallTree(W)
    r = 0.0000000001
    near = Array{Any}(undef, n)
    @inbounds @threads for k=1:n
        near[k] = NearestNeighbors.inrange(balltree, W[:,k], r, true)
    end
    near = map(sort,near) 
    @inbounds @threads for k=1:n
        W[:,k] = W[:,near[k][1]]
    end
    pointidx = Array{Int64}(undef, n)
    @inbounds @threads for k=1:n
         pointidx[k] = near[k][1] 
    end
    l = length(pointidx)
    invidx = OrderedDict(zip(1:l, pointidx))
    V = Array{Array{Float64,1}}(undef, l)
    @inbounds @threads for k=1:l
        V[k] = W[:,k] 
    end
    
    EV = []
    m = length(EW)
    @inbounds for i = 1:m
        newedge = [invidx[EW[i][1]],invidx[EW[i][2]]]
        if newedge[1] !== newedge[2]
            push!(EV,newedge)
        end
    end
    EV = [EV[h] for h=1:length(EV) if length(EV[h])==2]
    EV = convert(Lar.Cells, EV)
    return hcat(V...),EV
end


congruenceMOD (generic function with 1 method)

In [13]:
@btime congruenceMOD((V, EV))       #22,716 μs

  22.716 μs (176 allocations: 9.83 KiB)


([0.0 1.0 … 0.0 2.0; 0.0 0.0 … 1.0 1.0], [[1, 2], [2, 3], [3, 4], [4, 1], [1, 5]])

## TEST

In [11]:
using Test

@testset "congruence Tests" begin
    V1 = [0.0 1.0 ; 
         0.0 0.0]
    EV1 = [[1,2]]
    a, b = Lar.congruence((V1,EV1))
    @test a == V1
    @test b == EV1

    V1 = [0.0 1.0 0.99999999999999999 ; 0.0 0.0 0.0]
    EV1 = [[1,2], [1,3]]
    a, b = Lar.congruence((V1,EV1))
    @test b == [[1, 2], [1, 2]] 
    
end

Test Summary:    | Pass  Total
congruence Tests |    3      3


Test.DefaultTestSet("congruence Tests", Any[], 3, false, false)

![test di boxcovering](https://github.com/MarcoCap13/LARSplitting2D/blob/main/docs/test/congruence_test.png?raw=true)

### Benchmark della funzione iniziale e modificata

funzione iniziale:

In [14]:
@benchmark congruence((V, EV))   

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  36.619 μs …  36.981 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     38.992 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   78.811 μs ± 724.932 μs  ┊ GC (mean ± σ):  3.75% ± 1.00%

  █▄▂▂▁▂▂▃▂▁▁▁▁                                                ▁
  █████████████████▇▇█▇▇▇▇▆▇▆▆▆▆▅▆▆▅▅▅▅▆▅▅▅▅▄▅▄▅▃▅▄▁▄▄▃▄▄▃▃▄▅▅ █
  36.6 μs       Histogram: log(frequency) by time       362 μs <

 Memory estimate: 8.92 KiB, allocs estimate: 161.

funzione modificata:

In [15]:
@benchmark congruenceMOD((V, EV)) 

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  22.633 μs …   8.203 ms  ┊ GC (min … max): 0.00% … 99.33%
 Time  (median):     23.969 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   28.273 μs ± 112.349 μs  ┊ GC (mean ± σ):  5.53% ±  1.40%

  ██▆▄▃▁▁                                                      ▂
  ████████▇▇▇▆▆▆▆▆▆▇▇▇██▇█▆▆▆▆▆▆▅▄▆▅▅▅▅▅▅▄▆▄▅▅▃▅▆▄▃▄▄▄▄▅▄▃▄▁▄▄ █
  22.6 μs       Histogram: log(frequency) by time      90.7 μs <

 Memory estimate: 9.83 KiB, allocs estimate: 176.